# Notebook1 - Landing -> Bronze

## 1 - Configuração Inicial do Ambiente:

In [0]:
# Criar o Catálogo
spark.sql(f"CREATE CATALOG IF NOT EXISTS medalhao")
spark.sql(f"USE CATALOG medalhao")

# Criar os Schemas/Databases para as Camadas
spark.sql(f"CREATE SCHEMA IF NOT EXISTS bronze")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS silver")

# Criar o Volume
spark.sql(f"CREATE VOLUME IF NOT EXISTS default.landing")

- Define os nomes dos databases que serão usados
- Cria os databases bronze e silver se não existirem
- Usa o catalogo "medalhao" criado contendo todos os níveis(bronze, silver e gold)

## 2 - Função de Ingestão:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

# catalogo = "medalhao"
# bronze_db_name = "bronze" 

def ingest_csv(nome_arquivo_csv, nome_tabela_bronze):
    try:
        landing_path = f"/Volumes/medalhao/default/landing/{nome_arquivo_csv}"
        
        # Leitura do arquivo CSV
        df = spark.read.csv(landing_path, header=True, inferSchema=True)
        
        # Verifica se o arquivo não está vazio
        if df.count() == 0:
            raise ValueError(f"O arquivo {nome_arquivo_csv} está vazio.")

        # Adiciona a coluna 'ingestion_timestamp' com o timestamp atual
        df_with_metadata = df.withColumn("ingestion_timestamp", F.current_timestamp())

        # Escrita no formato Delta, sobrescrevendo se já existir
        df_with_metadata.write.format("delta").mode("overwrite").saveAsTable(f"bronze.{nome_tabela_bronze}")
        
        print(f"Tabela bronze.{nome_tabela_bronze} criada com sucesso a partir de {nome_arquivo_csv}")

    except Exception as e:
        print(f"Erro ao processar {nome_arquivo_csv}: {str(e)}")

- A função pode ser usada para a ingestão de todos os 9 arquivos CSV
- Verificação se o arquivo escolhido não está vazio
- Adição da coluna ingestion_timestamp
- Passagem para o formato Delta

## 3 - Ingestão de Cada um dos 9 arquivos:

In [0]:
# Mapeamento: Arquivo CSV -> Tabela Bronze
arquivos_e_tabelas = {
    "olist_customers_dataset.csv": "ft_consumidores",
    "olist_geolocation_dataset.csv": "ft_geolocalizacao",
    "olist_order_items_dataset.csv": "ft_itens_pedidos",
    "olist_order_payments_dataset.csv": "ft_pagamentos_pedidos",
    "olist_order_reviews_dataset.csv": "ft_avaliacoes_pedidos",
    "olist_orders_dataset.csv": "ft_pedidos",
    "olist_products_dataset.csv": "ft_produtos",
    "olist_sellers_dataset.csv": "ft_vendedores",
    "product_category_name_translation.csv": "dm_categoria_produtos_traducao"
}

# Execução da ingestão para todas as tabelas
for arquivo, tabela in arquivos_e_tabelas.items():
    ingest_csv(arquivo, tabela)

- O dicionário liga os arquivos CSV com os nomes das respectivas tabelas
- Loop para ingestão sequencial de todas as 9 tabelas

## 4 - Função para Ingerir Cotação do Dolar via API:

In [0]:
import requests
from datetime import datetime, timedelta

def ingest_cotacao_dolar(data_inicio_formatada, data_fim_formatada):
    # data_inicio_formatada no formato 'MM-DD-AAAA'
    # data_fim_formatada no formato 'MM-DD-AAAA'
    try:
        print(f"Ingerindo cotação dólar de {data_inicio_formatada} até {data_fim_formatada}")
        
        #URL da API
        url_api = (f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/"
                  f"CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)"
                  f"?@dataInicial='{data_inicio_formatada}'&@dataFinalCotacao='{data_fim_formatada}'"
                  f"&$select=dataHoraCotacao,cotacaoCompra,cotacaoVenda&$format=json")
        
        print(f"Período: {data_inicio_formatada} a {data_fim_formatada}")
        
        # Requisição à API
        response = requests.get(url_api, timeout=30)
        
        if response.status_code != 200:
            raise ValueError(f"Erro na API: Status {response.status_code} - {response.text}")
        
        dados = response.json()
        
        # Extrai lista de cotações
        cotacoes = dados.get('value', [])
        
        if not cotacoes:
            print("Nenhuma cotação encontrada no período")
            return False
        
        print(f"{len(cotacoes)} cotações recuperadas")
        
        # Mostra amostra dos dados
        if len(cotacoes) > 0:
            primeira_cotacao = cotacoes[0]
            print(f"Primeira cotação: {primeira_cotacao.get('dataHoraCotacao')} - R$ {primeira_cotacao.get('cotacaoCompra')}")
        
        # Converte para DataFrame Spark
        df_cotacoes = spark.createDataFrame(cotacoes)
        
        # Adiciona metadados
        df_cotacoes_com_metadados = (df_cotacoes
            .withColumn("ingestion_timestamp", F.current_timestamp())
            .withColumn("data_inicio_periodo", F.lit(data_inicio_formatada))
            .withColumn("data_fim_formatada_periodo", F.lit(data_fim_formatada))
        )
        
        # Mostra schema dos dados
        print("Schema da cotação:")
        df_cotacoes_com_metadados.printSchema()
        
        # Salva como tabela Delta
        (df_cotacoes_com_metadados
         .write
         .format("delta")
         .mode("overwrite")
         .saveAsTable(f"medalhao.bronze.dm_cotacao_dolar")
        )
        
        print(f"bronze.dm_cotacao_dolar criada com {df_cotacoes_com_metadados.count():,} registros")
        return True
        
    except Exception as e:
        print(f"Erro na cotação dólar: {str(e)}")
        return False

# Parâmetros podem ser alterados facilmente
data_inicio_formatada = "01-01-2016" # Primeira data do dataset
data_fim_formatada = "11-09-2025" # Data a qual eu fiz o código(aparentemente o dataset só vai até 2019)

print(f"Período da cotação: {data_inicio_formatada} a {data_fim_formatada}")

# Executa ingestão da cotação
ingestao_cotacao = ingest_cotacao_dolar(data_inicio_formatada, data_fim_formatada)

- A função realiza a consulta das cotações do dolar atravéz do endpoit da API do Banco Central
- As datas passadas como parâmetros da função seguem a formatação de MM/DD/AAAA
- Validação dos status da resposta e dados vazios
- Estrutura similar as outras tabelas criadas na camada bronze

## 5 - Validação Final da Camada Bronze:

In [0]:
# Lista todas as tabelas criadas no database bronze
tabelas_bronze = spark.sql("SHOW TABLES IN bronze")
tabelas_criadas = [row.tableName for row in tabelas_bronze.collect()]

print(f"Total de tabelas na Bronze: {len(tabelas_criadas)}")

for tabela in tabelas_criadas:
    # Conta registros em cada tabela
    contagem = spark.sql(f"SELECT COUNT(*) as total FROM bronze.{tabela}").collect()[0]['total']
    
    # Verifica se tem coluna de ingestion_timestamp
    colunas = spark.sql(f"DESCRIBE bronze.{tabela}")
    tem_timestamp = any("ingestion_timestamp" in row.col_name for row in colunas.collect())
    
    status_timestamp = "Yes" if tem_timestamp else "No"
    
    print(f"{status_timestamp}, {tabela}: {contagem:,} registros")

- Conta se possuem 10 tabelas na camada bronze, sendo as 9 importadas e 1 com as cotações do dólar
- Verifica a existencia da tabela 'ingestion_timestamp' em todas as tabelas da camada bronze